# 📈 Coleta de Séries via API — Fintech de Investimentos
**Notebook 03 — Indicadores (ultimo ano)**

Coleta e padroniza **indicadores financeiros** para enriquecer o banco analítico (DuckDB).
Fontes e séries:
- **Banco Central (SGS)**: SELIC (432), IPCA (433), CDI (12), Poupança (195), USD/BRL (1)
- **Yahoo Finance**: IBOVESPA (`^BVSP`)
- **CoinGecko**: Bitcoin (BTC/BRL)

Saídas em `data/raw/` (CSV/JSONL) e carga na tabela `indicadores`.

## 📦 Dependências
Instale (se necessário):
```bash
pip install pandas requests duckdb lxml html5lib tqdm yfinance
```


## ⚙️ Importações, Pastas e Janela de Datas (2 anos)

In [1]:
import os, json, time
from datetime import datetime, timedelta, timezone
import pandas as pd
import requests
from tqdm import tqdm

os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('banco', exist_ok=True)

DATA_FINAL = datetime.now()
DATA_INICIAL = DATA_FINAL - timedelta(days=365)

def fmt_bcb(d: datetime) -> str:
    return d.strftime('%d/%m/%Y')

print('Janela:', fmt_bcb(DATA_INICIAL), '→', fmt_bcb(DATA_FINAL))
print('✅ Ambiente pronto')


Janela: 24/10/2023 → 23/10/2025
✅ Ambiente pronto


## 🧰 Funções — Banco Central (SGS)

In [16]:
def coleta_sgs(serie_id: int, nome: str, data_inicial: datetime, data_final: datetime) -> pd.DataFrame:
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie_id}/dados"
    params = {
        "formato": "json",
        "dataInicial": fmt_bcb(data_inicial),
        "dataFinal": fmt_bcb(data_final)
    }
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    dados = r.json()

    if not dados:
        print(f"⚠️ Nenhum dado retornado para série {serie_id}")
        return pd.DataFrame(columns=['serie','nome','data','valor'])

    df = pd.DataFrame(dados)
    df = df.rename(columns={'data':'data','valor':'valor'})
    df['data'] = pd.to_datetime(df['data'], dayfirst=True).dt.date.astype(str)
    df['valor'] = df['valor'].astype(float)
    df.insert(0, 'serie', str(serie_id))
    df.insert(1, 'nome', nome)
    print(f"✅ Série {serie_id} ({nome}) — Linhas:", len(df))
    return df


In [ ]:
from datetime import datetime, timedelta

# intervalo de 2 anos
data_final = datetime.now()
data_inicial = data_final - timedelta(days=365)

df_selic = coleta_sgs(432, "SELIC (% a.a.)", data_inicial, data_final)

# Verifica o resultado
print("✅ Linhas retornadas:", len(df_selic))
display(df_selic.head(10))


## 💹 IBOVESPA — Yahoo Finance (`^BVSP`)

In [ ]:
!pip install yfinance

In [12]:
def coleta_ibov(data_inicial: datetime, data_final: datetime) -> pd.DataFrame:
    import yfinance as yf
    ticker = yf.Ticker('^BVSP')
    df = ticker.history(
        start=data_inicial.strftime('%Y-%m-%d'),
        end=data_final.strftime('%Y-%m-%d'),
        interval='1d'
    )
    if df.empty:
        print("⚠️ Nenhum dado retornado do Yahoo Finance.")
        return pd.DataFrame(columns=['serie','nome','data','valor'])
    df = df[['Close']].reset_index().rename(columns={'Date':'data','Close':'valor'})
    df['data'] = pd.to_datetime(df['data']).dt.date.astype(str)
    df['valor'] = df['valor'].astype(float)
    df.insert(0, 'serie', '^BVSP')
    df.insert(1, 'nome', 'IBOVESPA')
    return df


In [ ]:
df_ibov = coleta_ibov(DATA_INICIAL, DATA_FINAL)
print("✅ Linhas coletadas:", len(df_ibov))
df_ibov.head()

## ₿ Bitcoin (BTC/BRL) — CoinGecko

In [25]:
def coleta_btc_brl_coingecko(dias: int = 365) -> pd.DataFrame:
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {"vs_currency":"brl", "days": str(dias)}
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    prices = js.get('prices', [])
    if not prices:
        return pd.DataFrame(columns=['serie','nome','data','valor'])
    rows = []
    for ts_ms, price in prices:
        dt = datetime.fromtimestamp(ts_ms/1000.0, tz=datetime.UTC).date().isoformat()
        rows.append((dt, float(price)))
    df = pd.DataFrame(rows, columns=['data','valor']).groupby('data', as_index=False).last()
    df.insert(0, 'serie', 'BTCBRL')
    df.insert(1, 'nome', 'Bitcoin (BRL)')
    return df


In [ ]:
df_btc = coleta_btc_brl_coingecko()
print("📊 Linhas retornadas:", len(df_btc))
display(df_btc.head(10))

## ▶️ Coletando todas as séries (ultimo ano)

In [ ]:
from datetime import datetime, timedelta

DATA_FINAL = datetime.now()
DATA_INICIAL = DATA_FINAL - timedelta(days=365)

print("⏱ Intervalo definido:")
print("  DATA_INICIAL:", DATA_INICIAL.strftime("%Y-%m-%d"))
print("  DATA_FINAL:", DATA_FINAL.strftime("%Y-%m-%d"))


In [ ]:
dfs = []
series_bcb = [
    (432, 'SELIC (% a.a.)'),
    (433, 'IPCA (% mensal)'),
    (12,  'CDI diário (% a.a.)'),
    (195, 'Poupança (ref/%)'),
    (1,   'USD/BRL (venda)')
]

print("🚀 Iniciando coleta de séries do BCB...\n")

for sid, nome in series_bcb:
    try:
        print(f"➡️ Coletando {sid} - {nome} ...", end=" ")
        df = coleta_sgs(sid, nome, DATA_INICIAL, DATA_FINAL)
        print(f"{len(df)} linhas ✅")
        dfs.append(df)
    except Exception as e:
        print(f"❌ Falha em {sid} ({nome}):", e)

# IBOVESPA
try:
    df_ibov = coleta_ibov(DATA_INICIAL, DATA_FINAL)
    print(f"\n💹 IBOVESPA (^BVSP): {len(df_ibov)} linhas ✅")
    dfs.append(df_ibov)
except Exception as e:
    print("❌ Falha IBOV:", e)

# BITCOIN
try:
    df_btc = coleta_btc_brl_coingecko(dias=365)
    print(f"₿ BTC/BRL: {len(df_btc)} linhas ✅")
    dfs.append(df_btc)
except Exception as e:
    print("❌ Falha BTC:", e)

# Consolidação
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all[['serie', 'nome', 'data', 'valor']].sort_values(['nome', 'data']).reset_index(drop=True)
    print(f"\n📊 TOTAL de séries combinadas: {len(df_all)} linhas ✅")
    display(df_all.head(10))
else:
    print("\n⚠️ Nenhum dataset coletado.")

## 💾 Salvando CSV e JSONL (UTF-8)

In [ ]:
carimbo = datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')
csv_path = f'data/raw/series_{carimbo}.csv'
jsonl_path = f'data/raw/series_{carimbo}.jsonl'

df_all.to_csv(csv_path, index=False, encoding='utf-8')

with open(jsonl_path, 'w', encoding='utf-8') as f:
    for _, row in df_all.iterrows():
        f.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print('Arquivos salvos:')
print(' -', csv_path)
print(' -', jsonl_path)


## 🗄️ Inserção no DuckDB (`indicadores`)

In [ ]:
#Criar tabela Indicadores e inserir dados;
import duckdb
conn = duckdb.connect('banco/banco_analitico.duckdb')
conn.execute('''
CREATE TABLE IF NOT EXISTS indicadores (
    serie VARCHAR,
    nome  VARCHAR,
    data  DATE,
    valor DOUBLE
)
''')
conn.register('tmp_ind', df_all)
conn.execute("""
INSERT INTO indicadores
SELECT serie, nome, CAST(data AS DATE), CAST(valor AS DOUBLE)
FROM tmp_ind
""")
tot = conn.execute('SELECT COUNT(*) FROM indicadores').fetchone()[0]
amostra = conn.execute('SELECT * FROM indicadores ORDER BY data DESC LIMIT 5').fetchdf()
print('Total em indicadores:', tot)
amostra
conn.close()


In [ ]:
#Consultar dados da tabela indicadores;

import duckdb

# Caminho do banco
db_path = "banco/banco_analitico.duckdb"

# Conecta ao banco (modo leitura)
conn = duckdb.connect(db_path, read_only=True)

# Consulta básica
df_ind = conn.execute("""
    SELECT *
    FROM indicadores
    ORDER BY data DESC
    LIMIT 20
""").fetchdf()

print(f"🔎 Total de registros: {len(df_ind)}")
display(df_ind)

conn.close()
